# Setup

Follow the setup instructions in the [setup notebook](./setup.ipynb) before running this notebook. 

In [9]:
%matplotlib inline

In [10]:
import os
import mlflow
from datasets import load_dataset
from huggingface_hub import notebook_login
from matplotlib import pyplot as plt
from transformers import (
    AutoModelForSequenceClassification,
    RobertaTokenizer,
    pipeline
)

from mlflow import MlflowClient
from mlflow.store.artifact.runs_artifact_repo import RunsArtifactRepository

## Huggingface login
NOTE: you need create a huggingface account if you don't already have one and use it to generate a token in the next cell

In [3]:
notebook_login()

In [4]:
!git config --global credential.helper store

## Load Model and Tokenizer

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
classification = pipeline('text-classification', model=model, tokenizer=tokenizer)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Test model locally

In [12]:
text = "I am very excited today."
classification(text)

[{'label': 'positive', 'score': 0.9922448992729187}]

## Register model

In [14]:
model_name = "finance-sentiment-analysis"

client = MlflowClient()
client.create_registered_model(model_name)
with mlflow.start_run() as run:
    model_info = mlflow.transformers.log_model(classification, "model")
    runs_uri = model_info.model_uri
    desc = "Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text classifier"
    model_src = RunsArtifactRepository.get_underlying_uri(runs_uri)
    mv = client.create_model_version(model_name, model_src, run.info.run_id, description=desc)
    print("Name: {}".format(mv.name))
    print("Version: {}".format(mv.version))
    print("Description: {}".format(mv.description))
    print("Status: {}".format(mv.status))
    print("Stage: {}".format(mv.current_stage))

2024/09/13 21:16:38 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: finance-sentiment-analysis, version 1


Name: finance-sentiment-analysis
Version: 1
Description: Pretrained Sentiment Analysis model from mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis -- text classifier
Status: READY
Stage: None
